In [43]:
import pandas as pd
import numpy as np
from scipy.stats import poisson
from scipy.stats import geom
from scipy.stats import binom
from itertools import permutations
import math
from statistics import mean
import random
from scipy.stats import gamma
import matplotlib.pyplot as  plt
import copy
import scipy.stats


In [44]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


#Import Cases, Vaccination and AQI datasets (X dataset)

In [45]:
cases_df=pd.read_csv("/content/gdrive/MyDrive/United_States_COVID-19_Cases_and_Deaths_by_State_over_Time.csv")
vaccine_df=pd.read_csv("/content/gdrive/MyDrive/COVID-19_Vaccinations_in_the_United_States_Jurisdiction.csv")
aqi_2021=pd.read_csv("/content/gdrive/MyDrive/daily_aqi_by_county_2021.csv")
aqi_2020=pd.read_csv("/content/gdrive/MyDrive/daily_aqi_by_county_2020.csv")
vaccine_daily_de=pd.read_csv("/content/gdrive/MyDrive/delaware_vaccines.csv")
vaccine_daily_id=pd.read_csv("/content/gdrive/MyDrive/idaho_vaccines.csv")
co_2020 = pd.read_csv('/content/gdrive/MyDrive/daily_42101_2020.csv')
co_2021 = pd.read_csv('/content/gdrive/MyDrive/daily_42101_2021.csv')

#Exploratory

In [46]:
aqi=pd.concat([aqi_2020,aqi_2021])
aqi['Date'] = pd.to_datetime(aqi['Date'])
# aqi=aqi[aqi["State Name"]=="Idaho"]
aqi

,State Name,county Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2020-01-01,48,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2020-01-04,13,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2020-01-07,14,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2020-01-10,39,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2020-01-13,29,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
218191,Wyoming,Weston,56,45,2021-06-26,41,Good,Ozone,56-045-0003,2
218192,Wyoming,Weston,56,45,2021-06-27,40,Good,Ozone,56-045-0003,2
218193,Wyoming,Weston,56,45,2021-06-28,41,Good,Ozone,56-045-0003,2
218194,Wyoming,Weston,56,45,2021-06-29,48,Good,Ozone,56-045-0003,2


In [47]:
list(vaccine_daily_id["Date"][vaccine_daily_id["Series_Complete_Yes"]>480000])[0]

'2021-04-30'

In [48]:
co_2020['Date Local']=pd.to_datetime(co_2020['Date Local'])
# co_2020=co_2020.sort_values(by='Date Local')
co_2021['Date Local']=pd.to_datetime(co_2021['Date Local'])
# co_2021=co_2021.sort_values(by='Date Local')
co_2020

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
1,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
2,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
3,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
4,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179333,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,6.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-11-01
179334,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,9.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-11-01
179335,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,9.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-11-01
179336,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,6.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-11-01


In [49]:
co=pd.concat([co_2020,co_2021])
co['Date Local'] = pd.to_datetime(co['Date Local'])
# aqi=aqi[aqi["State Name"]=="Idaho"]
co

,State Code,County Code,Site Num,Parameter Code,POC,Latitude,Longitude,Datum,Parameter Name,Sample Duration,...,AQI,Method Code,Method Name,Local Site Name,Address,State Name,County Name,City Name,CBSA Name,Date of Last Change
0,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
1,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
2,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
3,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
4,1,73,23,42101,2,33.553056,-86.81500,WGS84,Carbon monoxide,1 HOUR,...,NaN,93.0,INSTRUMENTAL - GAS FILTER CORRELATION CO ANALYZER,North Birmingham,"NO. B'HAM,SOU R.R., 3009 28TH ST. NO.",Alabama,Jefferson,Birmingham,"Birmingham-Hoover, AL",2021-10-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106547,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,8.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-10-30
106548,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,7.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-10-30
106549,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,8.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-10-30
106550,72,127,3,42101,1,18.449814,-66.05251,WGS84,Carbon monoxide,8-HR RUN AVG END HOUR,...,7.0,NaN,-,San Juan,BALDORIOTY DE CASTRO AV.,Puerto Rico,San Juan,San Juan,"San Juan-Carolina-Caguas, PR",2021-10-30


In [50]:
co['AQI'] = co['AQI'].fillna(0)

In [51]:
co['AQI']

0         0.0
1         0.0
2         0.0
3         0.0
4         0.0
         ... 
106547    8.0
106548    7.0
106549    8.0
106550    7.0
106551    5.0
Name: AQI, Length: 285890, dtype: float64

In [52]:
state_to_id = {
'Alabama' : 'AL',
'Alaska' : 'AK',
'Arizona' : 'AZ',
'Arkansas' : 'AR',
'California' : 'CA',
'Colorado' : 'CO',
'Connecticut' : 'CT',
'Delaware' : 'DE',
'District of Columbia' : 'DC',
'Florida' : 'FL',
'Georgia' : 'GA',
'Hawaii' : 'HI',
'Idaho' : 'ID',
'Illinois' : 'IL',
'Indiana' : 'IN',
'Iowa' : 'IA',
'Kansas' : 'KS',
'Kentucky' : 'KY',
'Louisiana' : 'LA',
'Maine' : 'ME',
'Maryland' : 'MD',
'Massachusetts' : 'MA',
'Michigan' : 'MI',
'Minnesota' : 'MN',
'Mississippi' : 'MS',
'Missouri' : 'MO',
'Montana' : 'MT',
'Nebraska' : 'NE',
'Nevada' : 'NV',
'New Hampshire' : 'NH',
'New Jersey' : 'NJ',
'New Mexico' : 'NM',
'New York' : 'NY',
'North Carolina' : 'NC',
'North Dakota' : 'ND',
'Ohio' : 'OH',
'Oklahoma' : 'OK',
'Oregon' : 'OR',
'Pennsylvania' : 'PA',
'Rhode Island' : 'RI',
'South Carolina' : 'SC',
'South Dakota' : 'SD',
'Tennessee' : 'TN',
'Texas' : 'TX',
'Utah' : 'UT',
'Vermont' : 'VT',
'Virginia' : 'VA',
'Washington' : 'WA',
'West Virginia' : 'WV',
'Wisconsin' : 'WI',
'Wyoming' : 'WY'}

In [53]:
co_fin = co[['State Name', 'Date Local', 'AQI']].groupby(['State Name', 'Date Local']).agg({'AQI': 'mean'})
co_fin = co_fin.reset_index()

In [54]:
co_fin

,State Name,Date Local,AQI
0,Alabama,2020-01-01,2.666667
1,Alabama,2020-01-02,1.500000
2,Alabama,2020-01-03,2.166667
3,Alabama,2020-01-04,1.000000
4,Alabama,2020-01-05,2.333333
...,...,...,...
31017,Wyoming,2021-09-27,1.000000
31018,Wyoming,2021-09-28,1.000000
31019,Wyoming,2021-09-29,1.000000
31020,Wyoming,2021-09-30,0.500000


In [55]:
for d in state_to_id:
    co_fin = co_fin.replace(d,state_to_id[d])

In [56]:
co_fin

,State Name,Date Local,AQI
0,AL,2020-01-01,2.666667
1,AL,2020-01-02,1.500000
2,AL,2020-01-03,2.166667
3,AL,2020-01-04,1.000000
4,AL,2020-01-05,2.333333
...,...,...,...
31017,WY,2021-09-27,1.000000
31018,WY,2021-09-28,1.000000
31019,WY,2021-09-29,1.000000
31020,WY,2021-09-30,0.500000


In [57]:
co=co_fin[co_fin["State Name"]=="ID"]

We compare the 3 distributions before and after 4800000 double doses were completed. The AQI distribution of both periods, AQI cause by carbon monoxide distribution of both periods, AQI vs CO distribution of the before period and after period. We assume that after this amount of double doses were completed the lockdown was lifted. We check if this causes a change in AQI. We use KS test since we wanted to compare 2 disributions and it is applicable since it has no assumption


In [58]:
before_data_co=list(co["AQI"][(co["Date Local"]>='2020-05-01') & (co["Date Local"]<'2020-12-01')])
after_data_co=list(co["AQI"][(co["Date Local"]>=list(vaccine_daily_id["Date"][vaccine_daily_id["Series_Complete_Yes"]>480000])[0]) & (co["Date Local"]<'2021-12-31')])

In [59]:
def ks_test_2_sample(X1, Y1, X2, Y2):
    tot_max = -1
    ks_table = np.zeros((len(X1),6))
    for i in range(len(ks_table)-1):
        ks_table[i,0] = Y1[i]
        ks_table[i,1] = Y1[i+1]
        ks_table[i,2]=0
        ks_table[i,3]=0
        for j in X2:
            if j<X1[i]:
                ks_table[i,2]+=1
            if j<=X1[i]:
                ks_table[i,3]+=1

        ks_table[i,3]/=len(X2)
        ks_table[i,2]/=len(X2)

        ks_table[i,4] = abs(ks_table[i,0] - ks_table[i,2])
        ks_table[i,5] = abs(ks_table[i,1] - ks_table[i,3])
        cmax = max(ks_table[i,4], ks_table[i,5])
        if cmax > tot_max:
            tot_max = cmax
            x1_max = X1[i]
            y1_max = ks_table[i,0]
            y2_max = ks_table[i,2]
    if tot_max>0.05:
        print("We reject the null hypotheis that the 2 distributions follow the same distribution")
    else:
        print("We accept the null hypotheis that the 2 distributions follow the same distribution")
    return tot_max
    

In [60]:
def generate_eCDF(X):
  n = len(X)
  Srt = sorted(X)
  delta = .1
  X = [min(Srt)-delta]
  Y = [0]
  for i in range(0, n):
      X = X + [Srt[i], Srt[i]]
      Y = Y + [Y[len(Y)-1], Y[len(Y)-1]+(1/n)]
  X = X + [max(Srt)+delta]
  # print(X)
  Y = Y + [1]
  return X, Y

In [61]:
before_X, before_Y = generate_eCDF(before_data_co)
after_X, after_Y = generate_eCDF(after_data_co)
print(ks_test_2_sample(before_X, before_Y, after_X, after_Y))

We reject the null hypotheis that the 2 distributions follow the same distribution
0.24598422897196515


AQI

In [62]:
aqi_fin = aqi[['State Name', 'Date', 'AQI']].groupby(['State Name', 'Date']).agg({'AQI': 'mean'})
aqi_fin = aqi_fin.reset_index()

In [63]:
aqi_fin = aqi[['State Name', 'Date', 'AQI']].groupby(['State Name', 'Date']).agg({'AQI': 'mean'})
aqi_fin = aqi_fin.reset_index()

In [64]:
for d in state_to_id:
    aqi_fin = aqi_fin.replace(d,state_to_id[d])

In [65]:
aqi=aqi_fin[aqi_fin["State Name"]=="ID"]

In [66]:
before_data=list(aqi["AQI"][(aqi["Date"]>='2020-05-01') & (aqi["Date"]<'2020-12-01')])
after_data=list(aqi["AQI"][(aqi["Date"]>=list(vaccine_daily_id["Date"][vaccine_daily_id["Series_Complete_Yes"]>480000])[0]) & (aqi["Date"]<'2021-12-31')])

In [67]:
before_X, before_Y = generate_eCDF(before_data)
after_X, after_Y = generate_eCDF(after_data)
print(ks_test_2_sample(before_X, before_Y, after_X, after_Y))

We reject the null hypotheis that the 2 distributions follow the same distribution
0.2334338257461528


In [68]:
before_X, before_Y = generate_eCDF(before_data)
before_X_co, before_Y_co = generate_eCDF(before_data_co)
# after_X, after_Y = generate_eCDF(after_data)
print(ks_test_2_sample(before_X, before_Y, before_X_co, before_Y_co))

We reject the null hypotheis that the 2 distributions follow the same distribution
1.0


In [69]:
after_X, after_Y = generate_eCDF(after_data)
after_X_co, after_Y_co = generate_eCDF(after_data_co)
print(ks_test_2_sample(after_X, after_Y, after_X_co, after_Y_co))

We reject the null hypotheis that the 2 distributions follow the same distribution
1.0
